# YOLOv8 Licence Plate Character Train and Predict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf

In [ ]:
%pip install ultralytics

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from PIL import Image
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

# Check Annotation Data

In [ ]:
base_path = '/content/drive/My Drive/mining research/Faster_RCNN_Model/datasets'
ipaths = []
tpaths = []
for dirname, _, filenames in os.walk(base_path):
    for filename in filenames:
        if filename.endswith('jpeg'):
            ipaths.append(os.path.join(dirname, filename))
        elif filename.endswith('.txt'):
            tpaths.append(os.path.join(dirname, filename))
ipaths = sorted(ipaths)
tpaths = sorted(tpaths)
print(ipaths[0])
print(len(ipaths))

In [ ]:
boxdata=[]
boxfile=[]
for i in range(len(tpaths)):
    file=tpaths[i]
    boxdata+=[np.loadtxt(file)]
    boxfile+=[file[0:-4].split('/')[-1]]
print(boxdata[0])

In [ ]:
BOX=pd.DataFrame()

for i in range(len(boxdata)):
    if type(boxdata[i][0])==np.float64:
        add=pd.DataFrame([boxdata[i]])
        add[5]=boxfile[i]
        BOX=pd.concat([BOX,add])
    else:
        add=pd.DataFrame(boxdata[i])
        add[5]=boxfile[i]
        BOX=pd.concat([BOX,add])

BOX2=BOX.reset_index(drop=True)
display(BOX2)
print(BOX2.iloc[:,0].max())

In [ ]:
N=list(range(27))
Name=[]
for i in range(27):
    Name+=[str(i).zfill(2)]
class_map=dict(zip(N,Name))

In [ ]:
for i in range(len(BOX2)):
    BOX2.iloc[i,0]=class_map[int(BOX2.iloc[i,0])]
display(BOX2)
display(BOX2.iloc[:,0].value_counts())

In [ ]:
def draw_box(n0):

    ipath=ipaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath[0:-4].split('/')[-1]

    if BOX2[BOX2[5]==file] is not None:
        box=BOX2[BOX2[5]==file]
        box=box.reset_index(drop=True)

        for i in range(len(box)):
            label=box.iloc[i,0]
            x=box.iloc[i,1]
            y=box.iloc[i,2]
            w=box.iloc[i,3]
            h=box.iloc[i,4]
            x0=((x-w/2)*W).astype(int)
            y0=((y-h/2)*H).astype(int)
            x1=((x+w/2)*W).astype(int)
            y1=((y+h/2)*H).astype(int)
            cv2.rectangle(image,(x0,y0),(x1,y1),(0,255,0),1) #green

    #plt.imshow(image)
    #plt.show()

    return image

In [ ]:
images1=[]
for i in tqdm(range(len(ipaths))):#
    images1+=[draw_box(i)]

In [ ]:
# from matplotlib import animation, rc
# rc('animation', html='jshtml')

In [ ]:
# def create_animation(ims):
#     fig=plt.figure(figsize=(10,6))
#     #plt.axis('off')
#     im=plt.imshow(cv2.cvtColor(ims[0],cv2.COLOR_BGR2RGB))
#     plt.close()
#     def animate_func(i):
#         im.set_array(cv2.cvtColor(ims[i],cv2.COLOR_BGR2RGB))
#         return [im]
#     return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000//4)

In [ ]:
# create_animation(images1)

# Split Train, Valid and Test

In [ ]:
os.makedirs('datasets', exist_ok=True)
os.makedirs('datasets/train', exist_ok=True)
os.makedirs('datasets/valid', exist_ok=True)
os.makedirs('datasets/test', exist_ok=True)

In [ ]:
for i in range(len(ipaths)):
    ipath=ipaths[i]
    ifile=ipath.split('/')[-1]
    tpath=tpaths[i]
    tfile=tpath.split('/')[-1]
    if i%4==1:
        copyfile(ipath, f'datasets/valid/{ifile}')
        copyfile(tpath, f'datasets/valid/{tfile}')
    elif i%4==2:
        copyfile(ipath, f'datasets/test/{ifile}')
        #copyfile(tpath, f'datasets/test/{tfile}')
    else:
        copyfile(ipath, f'datasets/train/{ifile}')
        copyfile(tpath, f'datasets/train/{tfile}')

# Create yaml file

In [ ]:
import yaml

plate_yaml = dict(
    train ='train',
    val ='valid',
    test='test',
    nc =27,
    names = Name
)

with open('plate.yaml', 'w') as outfile:
    yaml.dump(plate_yaml, outfile, default_flow_style=True)

%cat plate.yaml

# Train

In [ ]:
model = YOLO("yolov8x.pt")

In [ ]:
!yolo task=detect mode=train model=yolov8x.pt data=plate.yaml epochs=40 imgsz=640

# Result of Training

In [ ]:
tpaths2=[]
for dirname, _, filenames in os.walk('/content/runs/detect/train'):
    for filename in filenames:
        if filename[-4:]=='.png' or filename[-4:]=='.jpg':
            tpaths2+=[(os.path.join(dirname, filename))]
tpaths2=sorted(tpaths2)
print(tpaths2[0])

In [ ]:
for path in tpaths2:
    image = Image.open(path)
    image=np.array(image)
    plt.figure(figsize=(20,10))
    plt.imshow(image)
    plt.show()

# Predict

In [ ]:
best_path0='/content/runs/detect/train/weights/best.pt'
source0='datasets/test'

In [ ]:
ppaths=[]
for dirname, _, filenames in os.walk(source0):
    for filename in filenames:
        if filename[-4:]=='jpeg':
            ppaths+=[(os.path.join(dirname, filename))]
ppaths=sorted(ppaths)
print(ppaths)

In [ ]:
model2 = YOLO(best_path0)

In [ ]:
!yolo task=detect mode=predict model={best_path0} conf=0.01 source={source0}

# Result of Prediction

In [ ]:
results = model2.predict(source0,conf=0.01)
print(len(results))

In [ ]:
print(results[0].boxes.data)

In [ ]:
PBOX=pd.DataFrame(columns=range(6))
for i in range(len(results)):#len(results)
    if len(results[i].boxes.data)>0:
        arri=pd.DataFrame(results[i].boxes.data.cpu().numpy()).astype(float)
        path=ppaths[i]
        file=path.split('/')[-1]
        arri=arri.assign(file=file)
        arri=arri.assign(i=i)
        PBOX=pd.concat([PBOX,arri],axis=0)
PBOX.columns=['x','y','x2','y2','confidence','class','file','i']
display(PBOX)

In [ ]:
PBOX['class']=PBOX['class'].apply(lambda x: class_map[int(x)])
PBOX=PBOX.reset_index(drop=True)
display(PBOX)
display(PBOX['class'].value_counts())

In [ ]:
color_codes = [
    '#FF0000',  # Red
    '#FF4500',  # OrangeRed
    '#FF8C00',  # DarkOrange
    '#FFA500',  # Orange
    '#FFD700',  # Gold
    '#FFFF00',  # Yellow
    '#ADFF2F',  # GreenYellow
    '#7FFF00',  # Chartreuse
    '#00FF00',  # Lime
    '#00FA9A',  # MediumSpringGreen
    '#00FFFF',  # Cyan
    '#00BFFF',  # DeepSkyBlue
    '#0000FF',  # Blue
    '#8A2BE2'   # BlueViolet
]
colors=[]
for c in color_codes:
    colors += [tuple((int(c[1:3],16),int(c[3:5],16),int(c[5:7],16)))]

In [ ]:
def draw_box2(n0):

    ipath=ppaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath.split('/')[-1]

    if PBOX[PBOX['file']==file] is not None:
        box=PBOX[PBOX['file']==file]
        box=box.reset_index(drop=True)
        #display(box)

        for i in range(len(box)):
            label=box.iloc[i,5]
            x=int(box.iloc[i,0])
            y=int(box.iloc[i,1])
            x2=int(box.iloc[i,2])
            y2=int(box.iloc[i,3])
            #print(label,x,y,x2,y2)
            ci=colors[int(label)%14]
            cv2.rectangle(image,(x,y),(x2,y2),ci,1)

    #plt.imshow(image)
    #plt.show()

    return image

In [ ]:
images2=[]
for i in tqdm(range(len(ppaths))):
    images2+=[draw_box2(i)]

In [ ]:
create_animation(images2)